# 第５章　誤差逆伝搬法

第4章では数値微分と勾配法を使って、ニューラルネットワークの学習が可能であることを示しました。実際の学習では計算時間および計算精度の観点から数値微分は使われずに、ニューラルネットワークの解析的微分を効率よく計算するためのアルゴリズムである誤差逆伝搬法が使われます。この章では誤差逆伝搬法を学習します。

## 単純な関数の解析的微分

単純な関数の微分は解析的な表現が知られていることが多いです。

$$
f(x) = x^n \Rightarrow \frac{df}{dx} = nx^{n-1} \\
f(x) = \sin(ax) \Rightarrow \frac{df}{dx} = a\cos(ax) \\
f(x) = \cos(ax) \Rightarrow \frac{df}{dx} = -a\sin(ax) \\
f(x) = \exp(ax) \Rightarrow \frac{df}{dx} = a\exp(ax)
$$


## 合成関数の微分

では、複雑な関数の微分はどのように計算するのでしょうか。ここで二つの関数
$$
y = f(x) \\
z = g(y)
$$
が与えられていると、その合成関数$g\circ f$を考えます。
$$
(g\circ f)(x) = g(f(x))
$$

多くの複雑な関数は単純な関数の合成で表現することができることに注意してください。
実は、合成関数$g\circ f$の微分は次のように、部品となった関数$f, g$の微分で表現することができます。

$$
    \frac{dz}{dx} = \frac{dz}{dy} \frac{dy}{dx}
$$

変数が複数ある場合は偏微分を使った次のような拡張が必要になります。例えば次のような関数を考えます。
$$
y_1 = f_1(x_1, x_2) \\
y_2 = f_2(x_1, x_2) \\
z = g(y_1, y_2)
$$
ここで、偏微分は次のように計算できることが知られています。
$$
\frac{\partial z}{\partial x_1} = \frac{\partial z}{\partial y_1}\frac{\partial y_1}{\partial x_1} + \frac{\partial z}{\partial y_2}\frac{\partial y_2}{\partial x_1}
$$
一般化して、ベクトルで表現すると、
$$
    {\bf y} = {\bf f}({\bf x}) \\
    z = g({\bf y})
$$
に対して、
$$
    \frac{\partial z}{\partial x_1} = \frac{\partial z}{\partial {\bf y}} \cdot \frac{\partial {\bf y}}{\partial x_1}
$$
と表現される

# ニューラルネットワークの場合

ニューラルネットワークは行列積と活性化層による非線形変換の組み合わせで表現されます。例えばニューラルネットワークの単一層は次のように表現されます。
$$
    {\bf a} = {\bf W}{\bf x} + {\bf B} \\
    {\bf z} = {\bf h}({\bf a})
$$
出力${\bf z}$の重みに対する微分は次のように表現できます。
$$
    \frac{\partial {\bf z}}{\partial{\bf W}} 
    = \frac{\partial {\bf z}}{\partial{\bf a}}\cdot \frac{\partial {\bf a}}{\partial{\bf W}} \\
    = {\bf h}'({\bf a})\cdot {\bf x}
$$